<a href="https://colab.research.google.com/github/maochaokuo/pyKerasGA1/blob/master/twfutures1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tw futures tendency prediction by keras
## place to imort


In [103]:
import pandas as pd
from collections import deque
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint
import time
from sklearn import preprocessing
from datetime import datetime, date, time
import datetime
import io
import sys
import pytz

tz = pytz.timezone('Asia/Taipei')
#sys.exit()
print(datetime.datetime.now(tz)) #Asia/Taipei
#PERCENT=0.2 # normaly 0.067 for a whole year
SEQ_LEN = 20  # how long of a preceeding sequence to collect for RNN
url='https://raw.githubusercontent.com/maochaokuo/tensorflow1/master/sharedfiles/aOHLCday.csv'
url2='https://raw.githubusercontent.com/maochaokuo/tensorflow1/master/sharedfiles/aOHLC.csv'

2019-08-17 09:55:32.268226+08:00


In [0]:
class Daily:
    def __init__ (self, ddate, dopen, dhigh, dlow, dclose, lastclose, hightime, lowtime):
        self.ddate=ddate
        self.dopen=dopen
        self.dhigh=dhigh
        self.dlow=dlow
        self.dclose=dclose
        self.lastclose=0
        self.hightime=84500
        self.lowtime=84500
    def __repr__(self):
        return '{ ddate: '+str(self.ddate)+', dopen: '+str(self.dopen)+', dhigh: '+str(self.dhigh)+', dlow: ' \
            +str(self.dlow)+', dclose: '+str(self.dclose)+', lastclose: '+str(self.lastclose)+', hightime: ' \
            +str(self.hightime)+', lowtime: '+str(self.lowtime)+' }'
    def __str__(self):
        return '{ ddate: '+str(self.ddate)+', dopen: '+str(self.dopen)+', dhigh: '+str(self.dhigh)+', dlow: ' \
            +str(self.dlow)+', dclose: '+str(self.dclose)+', lastclose: '+str(self.lastclose)+', hightime: ' \
            +str(self.hightime)+', lowtime: '+str(self.lowtime)+' }'
dailyList=[]
date2indexDict={}  

class ByMinute:
    def __init__ (self, ddate, dtime, dopen, dhigh, dlow, dclose, dvolume):
        self.ddate=ddate
        self.dtime=dtime
        self.dopen=dopen
        self.dhigh=dhigh
        self.dlow=dlow
        self.dclose=dclose
        self.dvolume=dvolume
    def __repr__(self):
        return '{ ddate: '+str(self.ddate)+', dtime: '+str(self.dtime)+', dopen: '+str(self.dopen)+', dhigh: ' \
            +str(self.dhigh)+', dlow: '+str(self.dlow)+', dclose: '+str(self.dclose)+', dvolume: ' \
            +str(self.dvolume)+' }' 
    def __str__(self):
        return '{ ddate: '+str(self.ddate)+', dtime: '+str(self.dtime)+', dopen: '+str(self.dopen)+', dhigh: ' \
            +str(self.dhigh)+', dlow: '+str(self.dlow)+', dclose: '+str(self.dclose)+', dvolume: ' \
            +str(self.dvolume)+' }' 
byMinuteList=[]      


In [105]:
from numpy import genfromtxt
npa = genfromtxt(url, dtype=float, delimiter=',', names=True)
npa

array([(20180629.,  84500., 10541., 10542., 10529., 10531., 4426.),
       (20180629.,  84600., 10531., 10532., 10526., 10531., 2142.),
       (20180629.,  84700., 10530., 10532., 10530., 10532.,  454.), ...,
       (20190719., 134200., 10733., 10737., 10732., 10737., 1014.),
       (20190719., 134300., 10736., 10741., 10735., 10740., 2862.),
       (20190719., 134400., 10740., 10745., 10737., 10744., 5158.)],
      dtype=[('dealdate', '<f8'), ('dealtime', '<f8'), ('open', '<f8'), ('high', '<f8'), ('low', '<f8'), ('close', '<f8'), ('volume', '<f8')])

In [106]:
#print(npa.shape[0])
#npa[0][0]

l=0
for i in npa:
    oneByMinute= ByMinute(i[0],i[1],i[2],i[3],i[4],i[5],i[6])
    '''
    oneByMinute.ddate=i[0]
    oneByMinute.dtime=i[1]
    oneByMinute.dopen=i[2]
    oneByMinute.dhigh=i[3]
    oneByMinute.dlow=i[4]
    oneByMinute.dclose=i[5]
    oneByMinute.dvolume=i[6]
    '''
    byMinuteList.append(oneByMinute)
    #print(i)
    #print(i[0])
    l=l+1
    #if l>10:
    #    break;
print(byMinuteList[0].ddate)
print(len(byMinuteList))

20180629.0
77864


In [0]:
#pd.DataFrame(data=data[1:,1:],    # values
#...              index=data[1:,0],    # 1st column as index
#...              columns=data[0,1:])  # 1st row as the column names
#npa[:, ['dealdate','dealtime','close']]
#npa['dealtime'][:5]
df=pd.DataFrame(npa)
#df

In [108]:
df['open'] = pd.to_numeric(df['open'], downcast='float')
df['high'] = pd.to_numeric(df['high'], downcast='float')
df['low'] = pd.to_numeric(df['low'], downcast='float')
df['close'] = pd.to_numeric(df['close'], downcast='float')
df['volume'] = pd.to_numeric(df['volume'], downcast='float')
print(df.dtypes)

dealdate    float64
dealtime    float64
open        float32
high        float32
low         float32
close       float32
volume      float32
dtype: object


In [0]:
df2=pd.DataFrame(df.groupby('dealdate')['dealdate'].last() )
df2=df2.join(df.groupby('dealdate')['open'].first())
df2=df2.join(df.groupby('dealdate')['high'].max())
df2=df2.join(df.groupby('dealdate')['low'].min())
df2=df2.join(df.groupby('dealdate')['close'].last())
df2['lastclose']=0
df2['hightime']=84500
df2['lowtime']=84500
#df2
#print(npa)
#print(df2.head())

npa2=df2.to_numpy()
#npa2


### 這邊更新完成dailyList與date2indexDict

In [110]:

#dailyList=[]
#date2indexDict={}  

l=0
for i in npa2:
    oneDaily= Daily(i[0],i[1],i[2],i[3],i[4],i[5],i[6],i[7])
    dailyList.append(oneDaily)
    '''
    print(i)
    print(i[0])
    l=l+1
    if l>10:
        break;
    '''
    date2indexDict[i[0]]=l
    l=l+1
print(len(dailyList))    
print(dailyList[:5])
#print(date2indexDict[0])
print(date2indexDict[20180629.0])
a=date2indexDict[20180702.0]
print(a)
print(dailyList[a])
#a[0:3]
#print(date2indexDict['20180629.0'].dopen)
#print(date2indexDict['20180629.0'].dhigh)
#2019/8/13做到這!!...

260
[{ ddate: 20180629.0, dopen: 10541.0, dhigh: 10692.0, dlow: 10526.0, dclose: 10677.0, lastclose: 0, hightime: 84500, lowtime: 84500 }, { ddate: 20180702.0, dopen: 10680.0, dhigh: 10719.0, dlow: 10609.0, dclose: 10611.0, lastclose: 0, hightime: 84500, lowtime: 84500 }, { ddate: 20180703.0, dopen: 10651.0, dhigh: 10717.0, dlow: 10544.0, dclose: 10622.0, lastclose: 0, hightime: 84500, lowtime: 84500 }, { ddate: 20180704.0, dopen: 10598.0, dhigh: 10639.0, dlow: 10584.0, dclose: 10618.0, lastclose: 0, hightime: 84500, lowtime: 84500 }, { ddate: 20180705.0, dopen: 10625.0, dhigh: 10633.0, dlow: 10488.0, dclose: 10500.0, lastclose: 0, hightime: 84500, lowtime: 84500 }]
0
1
{ ddate: 20180702.0, dopen: 10680.0, dhigh: 10719.0, dlow: 10609.0, dclose: 10611.0, lastclose: 0, hightime: 84500, lowtime: 84500 }


In [111]:
'''
df = pd.read_csv(url, names=['dealdate','dealtime','open','high', 'low', 'close', 'volume'])  # read in specific file

npa=df.values
print(npa.shape)
print(npa.dtype)
npa

df=df[1:]
print(df.head())
print(df.tail())
'''

"\ndf = pd.read_csv(url, names=['dealdate','dealtime','open','high', 'low', 'close', 'volume'])  # read in specific file\n\nnpa=df.values\nprint(npa.shape)\nprint(npa.dtype)\nnpa\n\ndf=df[1:]\nprint(df.head())\nprint(df.tail())\n"

In [112]:
print(df2.head())
print(df2.tail())

              dealdate     open     high  ...  lastclose  hightime  lowtime
dealdate                                  ...                              
20180629.0  20180629.0  10541.0  10692.0  ...          0     84500    84500
20180702.0  20180702.0  10680.0  10719.0  ...          0     84500    84500
20180703.0  20180703.0  10651.0  10717.0  ...          0     84500    84500
20180704.0  20180704.0  10598.0  10639.0  ...          0     84500    84500
20180705.0  20180705.0  10625.0  10633.0  ...          0     84500    84500

[5 rows x 8 columns]
              dealdate     open     high  ...  lastclose  hightime  lowtime
dealdate                                  ...                              
20190715.0  20190715.0  10808.0  10860.0  ...          0     84500    84500
20190716.0  20190716.0  10863.0  10892.0  ...          0     84500    84500
20190717.0  20190717.0  10837.0  10855.0  ...          0     84500    84500
20190718.0  20190718.0  10625.0  10676.0  ...          0     84500

In [113]:
npa2=df2.values
npa2

array([[2.0180629e+07, 1.0541000e+04, 1.0692000e+04, ..., 0.0000000e+00,
        8.4500000e+04, 8.4500000e+04],
       [2.0180702e+07, 1.0680000e+04, 1.0719000e+04, ..., 0.0000000e+00,
        8.4500000e+04, 8.4500000e+04],
       [2.0180703e+07, 1.0651000e+04, 1.0717000e+04, ..., 0.0000000e+00,
        8.4500000e+04, 8.4500000e+04],
       ...,
       [2.0190717e+07, 1.0837000e+04, 1.0855000e+04, ..., 0.0000000e+00,
        8.4500000e+04, 8.4500000e+04],
       [2.0190718e+07, 1.0625000e+04, 1.0676000e+04, ..., 0.0000000e+00,
        8.4500000e+04, 8.4500000e+04],
       [2.0190719e+07, 1.0757000e+04, 1.0792000e+04, ..., 0.0000000e+00,
        8.4500000e+04, 8.4500000e+04]])

In [114]:
aa=dailyList[0]
#print(aa)
aa

{ ddate: 20180629.0, dopen: 10541.0, dhigh: 10692.0, dlow: 10526.0, dclose: 10677.0, lastclose: 0, hightime: 84500, lowtime: 84500 }

In [115]:
l=1
for index, row in df2.iterrows():
    df2.at[index,'i']=int(l)
    l=l+1
lastclose=0
l=1
for index, row in df2.iterrows():
    dealdate=row['dealdate']
    idx=date2indexDict[dealdate]
    dailyRec=dailyList[idx]
    if lastclose==0:
        lastclose=row['close']
        if row['dealdate']==datetime.date(2018,6,29) or row['dealdate']=='20180629' or row['dealdate']==20180629.0:
            df2.at[index,'lastclose']=10523.0
            #print(len(dailyRec))
            dailyRec.lastclose=10523.0 #.lastclose
        continue
    df2.at[index,'lastclose']=lastclose
    dailyRec.lastclose=lastclose
    lastclose=row['close']
    l=l+1
print(datetime.datetime.now(tz)) # about 0.2 second
print(df2[['close','lastclose','i']].head())
print(df2[['close','lastclose','i']].tail())
for i in range(0, 5):
    print(dailyList[i])


2019-08-17 09:55:35.759869+08:00
              close  lastclose    i
dealdate                           
20180629.0  10677.0      10523  1.0
20180702.0  10611.0      10677  2.0
20180703.0  10622.0      10611  3.0
20180704.0  10618.0      10622  4.0
20180705.0  10500.0      10618  5.0
              close  lastclose      i
dealdate                             
20190715.0  10849.0      10810  256.0
20190716.0  10876.0      10849  257.0
20190717.0  10831.0      10876  258.0
20190718.0  10651.0      10831  259.0
20190719.0  10744.0      10651  260.0
{ ddate: 20180629.0, dopen: 10541.0, dhigh: 10692.0, dlow: 10526.0, dclose: 10677.0, lastclose: 10523.0, hightime: 84500, lowtime: 84500 }
{ ddate: 20180702.0, dopen: 10680.0, dhigh: 10719.0, dlow: 10609.0, dclose: 10611.0, lastclose: 10677.0, hightime: 84500, lowtime: 84500 }
{ ddate: 20180703.0, dopen: 10651.0, dhigh: 10717.0, dlow: 10544.0, dclose: 10622.0, lastclose: 10611.0, hightime: 84500, lowtime: 84500 }
{ ddate: 20180704.0, dopen: 1059

In [116]:
#date2indexDict
#dailyList[:5]
date2indexDict

{20180629.0: 0,
 20180702.0: 1,
 20180703.0: 2,
 20180704.0: 3,
 20180705.0: 4,
 20180706.0: 5,
 20180709.0: 6,
 20180710.0: 7,
 20180711.0: 8,
 20180712.0: 9,
 20180713.0: 10,
 20180716.0: 11,
 20180717.0: 12,
 20180718.0: 13,
 20180719.0: 14,
 20180720.0: 15,
 20180723.0: 16,
 20180724.0: 17,
 20180725.0: 18,
 20180726.0: 19,
 20180727.0: 20,
 20180730.0: 21,
 20180731.0: 22,
 20180801.0: 23,
 20180802.0: 24,
 20180803.0: 25,
 20180806.0: 26,
 20180807.0: 27,
 20180808.0: 28,
 20180809.0: 29,
 20180810.0: 30,
 20180813.0: 31,
 20180814.0: 32,
 20180815.0: 33,
 20180816.0: 34,
 20180817.0: 35,
 20180820.0: 36,
 20180821.0: 37,
 20180822.0: 38,
 20180823.0: 39,
 20180824.0: 40,
 20180827.0: 41,
 20180828.0: 42,
 20180829.0: 43,
 20180830.0: 44,
 20180831.0: 45,
 20180903.0: 46,
 20180904.0: 47,
 20180905.0: 48,
 20180906.0: 49,
 20180907.0: 50,
 20180910.0: 51,
 20180911.0: 52,
 20180912.0: 53,
 20180913.0: 54,
 20180914.0: 55,
 20180917.0: 56,
 20180918.0: 57,
 20180919.0: 58,
 201809

## orignal data to process (predict the tendency)

In [117]:
df.dtypes

dealdate    float64
dealtime    float64
open        float32
high        float32
low         float32
close       float32
volume      float32
dtype: object

In [118]:

#df = pd.read_csv(url, names=['dealdate','dealtime','open','high', 'low', 'close', 'volume'])  # read in specific file

'''
dfall = pd.read_csv(url2, names=['dealdate','dealtime','open','high', 'low', 'close', 'volume'])  # read in specific file
'''
#cols=["dealdate","dealtime","open","high", "low", "close", "volume"]
#df = pd.read_csv(url, names=cols, dtype=dict(zip(cols,[str, str, float, float, float, float, int])) ) 
#df.info()

#df['dealdate']=df['dealdate'].astype('|S')

#df.dealdate = df.dealdate.astype(str)
#df.dealtime = df.dealtime.astype(str)
#df.open=df.open.astype(float)
#df['dealtime']='0'+df['dealtime']
#df['dealtime']=df['dealtime'][-6:]
df=df.iloc[1:] #remove 1st row, the header
#dfall=dfall.iloc[1:]

#dropIndexes=df[ df['dealdate']>'20180731' ].index
#df.drop(dropIndexes, inplace=True)
'''
for index, row in df.iterrows():
    dealdate=row['dealdate']#[:4]+'-'+row['dealdate'][4:6]+'-'+row['dealdate'][6:]
    #df.at[index,'dealdate']=datetime.datetime.strptime(dealdate, '%Y%m%d')
    #df.at[index,'dealtime']=datetime.datetime.strptime(row['dealtime'], '%H%M%S')
    df.at[index,'dealdatetime']=row['dealdate'][:4]+'-'+row['dealdate'][4:6]+'-'+row['dealdate'][6:]+' '+row['dealtime'][:2]+':'+row['dealtime'][2:4]+':'+row['dealtime'][4:]
'''    
#df['dealdatetime']= pd.to_datetime(df['dealdatetime'])
#df['dealtime']=df['dealdatetime'].dt.time
#df['dealdate']=df['dealdatetime'].dt.date

'''
dfmorning=dfall.copy()
dropIndexes=dfmorning[ dfmorning['dealtime']>'060000' ].index
dfmorning.drop(dropIndexes, inplace=True)

dfnight=dfall.copy()
dropIndexes=dfnight[ dfnight['dealtime']<'140000'].index
dfnight.drop(dropIndexes, inplace=True)
'''
#df.groupby('dealdate')['open'].first()
#df.groupby('dealdate')['close'].last()
#df.groupby('dealdate')['low'].min()
#df.groupby('dealdate')['high'].max()
print(datetime.datetime.now(tz)) # about 0.5 second
#df['dealdate'].dtype


2019-08-17 09:55:35.845865+08:00


In [119]:
#dfNext['dealdate']=str(dfNext['dealdate'])
dfNext=df
'''
for index, row in dfNext.iterrows():
    dealdate=row['dealdate']#[:4]+'-'+row['dealdate'][4:6]+'-'+row['dealdate'][6:]
    #dfNext.at[index,'dealdate']=row['dealdate'] #datetime.datetime.strptime(dealdate, '%Y%m%d')
    #dfNext.at[index,'dealtime']=row['dealtime'] #datetime.datetime.strptime(row['dealtime'], '%H%M%S')
    #dfNext.at[index,'dealtime']=datetime.datetime.strptime(dealdate[:4]+'-'+dealdate[4:6]+'-'+dealdate[6:8])
    #dfNext.at[index,'dealdate']=row['dealdate'][:4]+'-'+row['dealdate'][4:6]+'-'+row['dealdate'][6:]
    #dfNext.at[index,'dealtime']=row['dealtime'][:2]+':'+row['dealtime'][2:4]+':'+row['dealtime'][4:]
    dfNext.at[index,'dealdatetime']=row['dealdate'][:4]+'-'+row['dealdate'][4:6]+'-'+row['dealdate'][6:]+' '+row['dealtime'][:2]+':'+row['dealtime'][2:4]+':'+row['dealtime'][4:]
dfNext['dealdatetime']= pd.to_datetime(dfNext['dealdatetime'])
dfNext['dealtime']=dfNext['dealdatetime'].dt.time
dfNext['dealdate']=dfNext['dealdatetime'].dt.date
'''

#dfNext['dealtime']= pd.to_datetime(dfNext['dealdatetime']).time()
print(dfNext.head())
print(dfNext.dtypes)
#wrong!!!!!!!!!!!!!!!!!!

     dealdate  dealtime     open     high      low    close  volume
1  20180629.0   84600.0  10531.0  10532.0  10526.0  10531.0  2142.0
2  20180629.0   84700.0  10530.0  10532.0  10530.0  10532.0   454.0
3  20180629.0   84800.0  10531.0  10538.0  10529.0  10536.0  1204.0
4  20180629.0   84900.0  10537.0  10537.0  10527.0  10527.0  1522.0
5  20180629.0   85000.0  10527.0  10532.0  10526.0  10531.0  1308.0
dealdate    float64
dealtime    float64
open        float32
high        float32
low         float32
close       float32
volume      float32
dtype: object


In [120]:
'''
for index, row in dfNext.iterrows():
    dfNext.at[index,'dealdateI']=int(row['dealdatetime'].strftime("%Y%m%d"))
    dfNext.at[index,'dealtimeI']=int(row['dealdatetime'].strftime("%H%M%S"))
dfNext.head()
'''

'\nfor index, row in dfNext.iterrows():\n    dfNext.at[index,\'dealdateI\']=int(row[\'dealdatetime\'].strftime("%Y%m%d"))\n    dfNext.at[index,\'dealtimeI\']=int(row[\'dealdatetime\'].strftime("%H%M%S"))\ndfNext.head()\n'

In [121]:

df.dtypes
#df1=df(['dealdate','dealtime','open','high','low','close','volume'], dtypes=[np.str,np.str,np.float64,np.float64,np.float64,np.float64,np.int64])
#df.dtypes
df.head()

,dealdate,dealtime,open,high,low,close,volume
1,20180629.0,84600.0,10531.0,10532.0,10526.0,10531.0,2142.0
2,20180629.0,84700.0,10530.0,10532.0,10530.0,10532.0,454.0
3,20180629.0,84800.0,10531.0,10538.0,10529.0,10536.0,1204.0
4,20180629.0,84900.0,10537.0,10537.0,10527.0,10527.0,1522.0
5,20180629.0,85000.0,10527.0,10532.0,10526.0,10531.0,1308.0


## df2 whole day summary (#1)

## 我們保留df2 dataframe groupby
剩下的都轉numpy做做看 undone !!... here

In [122]:
npdf = df.values
npdf.shape
npdf


array([[2.0180629e+07, 8.4600000e+04, 1.0531000e+04, ..., 1.0526000e+04,
        1.0531000e+04, 2.1420000e+03],
       [2.0180629e+07, 8.4700000e+04, 1.0530000e+04, ..., 1.0530000e+04,
        1.0532000e+04, 4.5400000e+02],
       [2.0180629e+07, 8.4800000e+04, 1.0531000e+04, ..., 1.0529000e+04,
        1.0536000e+04, 1.2040000e+03],
       ...,
       [2.0190719e+07, 1.3420000e+05, 1.0733000e+04, ..., 1.0732000e+04,
        1.0737000e+04, 1.0140000e+03],
       [2.0190719e+07, 1.3430000e+05, 1.0736000e+04, ..., 1.0735000e+04,
        1.0740000e+04, 2.8620000e+03],
       [2.0190719e+07, 1.3440000e+05, 1.0740000e+04, ..., 1.0737000e+04,
        1.0744000e+04, 5.1580000e+03]])

In [123]:
print(df.head())
print(df2.head())
print(df.dtypes)
print(df2.dtypes)

     dealdate  dealtime     open     high      low    close  volume
1  20180629.0   84600.0  10531.0  10532.0  10526.0  10531.0  2142.0
2  20180629.0   84700.0  10530.0  10532.0  10530.0  10532.0   454.0
3  20180629.0   84800.0  10531.0  10538.0  10529.0  10536.0  1204.0
4  20180629.0   84900.0  10537.0  10537.0  10527.0  10527.0  1522.0
5  20180629.0   85000.0  10527.0  10532.0  10526.0  10531.0  1308.0
              dealdate     open     high  ...  hightime  lowtime    i
dealdate                                  ...                        
20180629.0  20180629.0  10541.0  10692.0  ...     84500    84500  1.0
20180702.0  20180702.0  10680.0  10719.0  ...     84500    84500  2.0
20180703.0  20180703.0  10651.0  10717.0  ...     84500    84500  3.0
20180704.0  20180704.0  10598.0  10639.0  ...     84500    84500  4.0
20180705.0  20180705.0  10625.0  10633.0  ...     84500    84500  5.0

[5 rows x 9 columns]
dealdate    float64
dealtime    float64
open        float32
high        float32


In [124]:
dailyList[0]

{ ddate: 20180629.0, dopen: 10541.0, dhigh: 10692.0, dlow: 10526.0, dclose: 10677.0, lastclose: 10523.0, hightime: 84500, lowtime: 84500 }

## df2 whole day summary (#2)

In [131]:
for index, row in df2.iterrows():
    idx=date2indexDict[row['dealdate']]
    tmpRec=dailyList[idx]
    #print(tmpRec)
    df3=df[(df['dealdate']==row['dealdate']) & (df['high']==row['high'])] #first dealtime when high hit highest
    #row['hightime']=df3['dealtime'].iloc[0]
    df2.at[index,'hightime']=df3['dealtime'].iloc[0]
    tmpRec.hightime=df3['dealtime'].iloc[0]
    df3=df[(df['dealdate']==row['dealdate']) & (df['low']==row['low'])] #first dealtime when low hit lowest
    df2.at[index,'lowtime']=df3['dealtime'].iloc[0]
    tmpRec.lowtime=df3['dealtime'].iloc[0]
    #row['hightime']=
    #df3.iloc[0]['hightime']  #df[['dealtime']][ (df['dealdate']==row['dealdate']) & (df['high']==row['high'])]
    #print(tmp)
    #break
#df3= df[(df['dealdate']=='20180629') & (df['high']=='10692.00')] 
#df3.iloc[0]
print(datetime.datetime.now(tz)) # about 5 seconds
df2.head()
for i in range(0,5):
    print(dailyList[i])
#df2.tail()

2019-08-17 10:02:20.572034+08:00
{ ddate: 20180629.0, dopen: 10541.0, dhigh: 10692.0, dlow: 10526.0, dclose: 10677.0, lastclose: 10523.0, hightime: 132900.0, lowtime: 84600.0 }
{ ddate: 20180702.0, dopen: 10680.0, dhigh: 10719.0, dlow: 10609.0, dclose: 10611.0, lastclose: 10677.0, hightime: 101500.0, lowtime: 134300.0 }
{ ddate: 20180703.0, dopen: 10651.0, dhigh: 10717.0, dlow: 10544.0, dclose: 10622.0, lastclose: 10611.0, hightime: 92500.0, lowtime: 112100.0 }
{ ddate: 20180704.0, dopen: 10598.0, dhigh: 10639.0, dlow: 10584.0, dclose: 10618.0, lastclose: 10622.0, hightime: 93800.0, lowtime: 111700.0 }
{ ddate: 20180705.0, dopen: 10625.0, dhigh: 10633.0, dlow: 10488.0, dclose: 10500.0, lastclose: 10618.0, hightime: 84500.0, lowtime: 131300.0 }


In [0]:
df2.dtypes

接下來算大趨勢

!!!>>>>> 目前這裡非常慢

In [0]:
print(df.dtypes)
print(df2.dtypes)
print(df.head())
#print(df2.head())

l=0
#for index, row in df2.iterrows():
#    ...
#df2=df2.drop(["dealdate","i"]) #, axis=1)
print(df2.head())

In [0]:
df.head()

In [0]:
#df.head(10)
#print(datetime.datetime.now(tz))    # about 1:22 seconds, or 82 seconds
print(datetime.datetime.now(tz).strftime("%m/%d/%Y, %H:%M:%S"))
daterow=None
curdate=20000101 #datetime.datetime(2000, 1, 1).date() #'20000101'
l=0
for index, row in df.iterrows():
    dealdate=row['dealdate']
    dealtime=row['dealtime']
    # daily summary
    if daterow is None or curdate<dealdate:
        #就是這裡慢
        idx=date2indexDict[dealdate]
        dailyRec=dailyList[idx]
        datehightime=dailyRec.hightime
        datelowtime=dailyRec.lowtime
        ''' 
        daterow=df2[ df2['dealdate']==dealdate ].iloc[0] 
        datehightime=daterow['hightime']
        datelowtime=daterow['lowtime']
        ''' 
    else:
        curdate=dealdate
    if datehightime < datelowtime: # high earlier than low
        if dealtime < datehightime:
            df.at[index,'tendMajor']=1
        elif dealtime >= datehightime and dealtime < datelowtime:
            df.at[index,'tendMajor']=-1
        else:
            df.at[index,'tendMajor']=1
    else: # low earlier than high
        if dealtime < datelowtime:
            df.at[index,'tendMajor']=-1
        elif dealtime >= datelowtime and dealtime < datehightime:
            df.at[index,'tendMajor']=1
        else:
            df.at[index,'tendMajor']=-1
    l=l+1
print('l='+str(l))
#print(datetime.datetime.now(tz))    # about 1:22 seconds, or 82 seconds
print(datetime.datetime.now(tz).strftime("%m/%d/%Y, %H:%M:%S"))
#df.head(500)

In [142]:
#df[240:300]

,dealdate,dealtime,open,high,low,close,volume,tendMajor
241,20180629.0,124600.0,10672.0,10674.0,10670.0,10673.0,512.0,1.0
242,20180629.0,124700.0,10672.0,10673.0,10670.0,10671.0,360.0,1.0
243,20180629.0,124800.0,10671.0,10673.0,10670.0,10673.0,264.0,1.0
244,20180629.0,124900.0,10672.0,10675.0,10671.0,10674.0,318.0,1.0
245,20180629.0,125000.0,10674.0,10678.0,10673.0,10678.0,836.0,1.0
246,20180629.0,125100.0,10677.0,10680.0,10677.0,10678.0,912.0,1.0
247,20180629.0,125200.0,10679.0,10680.0,10678.0,10680.0,436.0,1.0
248,20180629.0,125300.0,10679.0,10683.0,10679.0,10679.0,1206.0,1.0
249,20180629.0,125400.0,10679.0,10680.0,10678.0,10679.0,352.0,1.0
250,20180629.0,125500.0,10679.0,10680.0,10675.0,10677.0,1012.0,1.0


In [145]:

#df.to_csv('dfTendMajor.csv', header=True, index=False)

print(df.head(10))
print(df.dtypes)

      dealdate  dealtime     open  ...  volume  tendMajor  tendMinor
1   20180629.0   84600.0  10531.0  ...  2142.0        1.0          0
2   20180629.0   84700.0  10530.0  ...   454.0        1.0          0
3   20180629.0   84800.0  10531.0  ...  1204.0        1.0          0
4   20180629.0   84900.0  10537.0  ...  1522.0        1.0          0
5   20180629.0   85000.0  10527.0  ...  1308.0        1.0          0
6   20180629.0   85100.0  10531.0  ...   888.0        1.0          0
7   20180629.0   85200.0  10533.0  ...   506.0        1.0          0
8   20180629.0   85300.0  10530.0  ...   668.0        1.0          0
9   20180629.0   85400.0  10528.0  ...   608.0        1.0          0
10  20180629.0   85500.0  10532.0  ...   522.0        1.0          0

[10 rows x 9 columns]
dealdate     float64
dealtime     float64
open         float32
high         float32
low          float32
close        float32
volume       float32
tendMajor    float64
tendMinor      int64
dtype: object


接下來算小趨勢

要改為用cTop, cBottom, 可以不用大趨勢，要smooth到5分鐘

可能用excel比較快

In [147]:

print(datetime.datetime.now(tz)) 
curdate=20101010 #datetime.datetime(2000, 1, 1).date() #'20000101'
pre2index=-1
preindex=-1
pre2high=-1.0
pre2low=-1.0
prehigh=-1.0
prelow=-1.0
l=0
df['tendMinor']=0
for index, row in df.iterrows():
    if row['dealdate']=='dealdate':
        continue
    if row['dealdate'] > curdate: ## change date
        pre2index = -1
        preindex = -1 
        pre2high=-1.0
        pre2low=-1.0
        prehigh=-1.0
        prelow=-1.0
    curdate=row['dealdate']
    tendMinor = 0
    #print(pre2high)
    #print(pre2low)
    #print(prehigh)
    #print(prelow)
    if pre2high>0 and pre2low>0 and prehigh>0 and prelow>0:
        if prehigh > pre2high and prehigh >float(row['high']): #local high at prerow
            tendMinor = -1
        if prelow < pre2low and prelow <float( row['low']): #local low at prerow
            if tendMinor!=0:
                tendMinor = 0
            else:
                tendMinor = 1
    #print('1 preindex='+str(preindex))
    if preindex>0:
        #print('prerow not none')
        df.at[preindex,'tendMinor']=tendMinor
    #else:
    #  print('prerow none')
    # before end of for
    pre2index = preindex
    preindex = index
    #print('2 preindex='+str(preindex))
    pre2high=prehigh
    pre2low=prelow
    try:
        prehigh=float(row['high'])
        prelow=float(row['low'])
    except:
        print(row)
        break
    l=l+1
    #if l>300:
    #  break
print('l='+str(l))
print(datetime.datetime.now(tz))  # about 14 seconds
df.head(10)

2019-08-17 10:17:25.456875+08:00
l=77863
2019-08-17 10:17:38.287622+08:00


,dealdate,dealtime,open,high,low,close,volume,tendMajor,tendMinor
1,20180629.0,84600.0,10531.0,10532.0,10526.0,10531.0,2142.0,1.0,0
2,20180629.0,84700.0,10530.0,10532.0,10530.0,10532.0,454.0,1.0,0
3,20180629.0,84800.0,10531.0,10538.0,10529.0,10536.0,1204.0,1.0,-1
4,20180629.0,84900.0,10537.0,10537.0,10527.0,10527.0,1522.0,1.0,0
5,20180629.0,85000.0,10527.0,10532.0,10526.0,10531.0,1308.0,1.0,1
6,20180629.0,85100.0,10531.0,10535.0,10529.0,10532.0,888.0,1.0,-1
7,20180629.0,85200.0,10533.0,10534.0,10530.0,10530.0,506.0,1.0,0
8,20180629.0,85300.0,10530.0,10531.0,10528.0,10528.0,668.0,1.0,0
9,20180629.0,85400.0,10528.0,10533.0,10528.0,10533.0,608.0,1.0,0
10,20180629.0,85500.0,10532.0,10533.0,10529.0,10530.0,522.0,1.0,0


In [152]:
df.head(60)

,dealdate,dealtime,open,high,low,close,volume,tendMajor,tendMinor
1,20180629.0,84600.0,10531.0,10532.0,10526.0,10531.0,2142.0,1.0,0
2,20180629.0,84700.0,10530.0,10532.0,10530.0,10532.0,454.0,1.0,0
3,20180629.0,84800.0,10531.0,10538.0,10529.0,10536.0,1204.0,1.0,-1
4,20180629.0,84900.0,10537.0,10537.0,10527.0,10527.0,1522.0,1.0,0
5,20180629.0,85000.0,10527.0,10532.0,10526.0,10531.0,1308.0,1.0,1
6,20180629.0,85100.0,10531.0,10535.0,10529.0,10532.0,888.0,1.0,-1
7,20180629.0,85200.0,10533.0,10534.0,10530.0,10530.0,506.0,1.0,0
8,20180629.0,85300.0,10530.0,10531.0,10528.0,10528.0,668.0,1.0,0
9,20180629.0,85400.0,10528.0,10533.0,10528.0,10533.0,608.0,1.0,0
10,20180629.0,85500.0,10532.0,10533.0,10529.0,10530.0,522.0,1.0,0


這裡試算損益以決定最佳趨勢

In [0]:
def calculatePL(): # calculate PL
    curdate=datetime.datetime(2000, 1, 1).date() #'20000101'
    pretend=0
    baseprice=0
    inventoryPLmin=0
    settlePLmin=0
    settlePLsum=0
    for index, row in df.iterrows():
        if row['dealdate']>curdate:
            curdate=row['dealdate']
            pretend=0
            baseprice=0
        dealtime=row['dealtime']
        close=float(row['close'])
        earliest=datetime.datetime.now(tz).replace(hour=9, minute=5, second=0, microsecond=0).time()
        latest=datetime.datetime.now(tz).replace(hour=13, minute=35, second=0, microsecond=0).time()
        #print(earliest)
        #print(latest)
        if row['dealtime'] < earliest or row['dealtime'] > latest :
            curtend=0
        else:
            tendMajor=int(df.at[index, 'tendMajor'])
            if np.isnan( df.at[index, 'tendMinor'] ):
                print('wrong! '+curdate+' '+dealtime)
            tendMinor=int(df.at[index, 'tendMinor'])
            tendMinor=int(df.at[index, 'tendMinor'])
            if tendMinor!=0:
                curtend=tendMinor
            else:
                #curtend=pretend
                if pretend!=0:
                    curtend=pretend
                else:
                    curtend=tendMajor
        inventoryPL=0
        settlePL=0
        if curtend!=pretend:
            if pretend!=0 and baseprice==0:
                print('wrong! '+curdate+' '+dealtime)
                break
            settlePL=pretend*(close-baseprice)
            if settlePL<settlePLmin:
                settlePLmin = settlePL
            settlePLsum=settlePLsum+settlePL
            if curtend==0:
                baseprice=0
            else:
                baseprice=close
        else: #curtend==pretend
            inventoryPL=curtend*(close-baseprice)
            if inventoryPL<inventoryPLmin:
                inventoryPLmin = inventoryPL
        df.at[index, 'testTend']=curtend
        df.at[index, 'baseprice']=baseprice
        df.at[index, 'inventoryPL']=inventoryPL
        df.at[index, 'inventoryPLmin']=inventoryPLmin
        df.at[index, 'settlePL']=settlePL
        df.at[index, 'settlePLmin']=settlePLmin
        df.at[index, 'settlePLsum']=settlePLsum
        df.at[index, 'index']=index
        pretend=curtend
calculatePL()        
print(df.drop(["open","high","low","volume"], axis=1).tail(10))
#df[ df['dealdate']=='20180702']    
print(datetime.datetime.now(tz))  # about 14 seconds

In [0]:
df.drop(["open","high","low","volume"], axis=1).tail(10)

# 瓶頸
寫到這裡，想要把趨勢再更準一點，因為等於只是提早一分鐘知道小趨勢而已
pandas超難用，型態轉換有問題，煩死了。看看能否換用numpy, 不然就放棄python, 改用rust寫
不過不行的是，免費的google colab只有python, 哀

## 暫時結論
發現真的只要早一分鐘，盈虧就差很多，
再多做一件，若次筆趨勢完全相反，這筆趨勢就不算，這樣避免掉不少反覆

差一分鐘差之千里。再多做反覆smooth out, 最好可以五分鐘內不反覆

In [0]:
df.loc[df.index==20793,'testTend']=1
df[df['index']==20793].tail()
'''
df[df['index']==20793]['testTend']=-df[df['index']==20793]['testTend']
print(df[df['index']==20793]['testTend'])
'''

In [0]:
dftmp0=df[df['inventoryPLmin']==-84].head(1)
print(dftmp0)
#print(df[df['inventoryPLmin']==-84].head(1))
#print(df['dealdate'].dtype)
#df2=pd.DataFrame( df.groupby('dealdate')['open'].first() )
dftmp1=df.drop(["open","high","low","volume",'dealdateI','dealtimeI','testTend','baseprice'], axis=1).tail()
dftmp1 #[dftmp1['index']==20793]
#dftmp1[dftmp1['dealdate']==dftmp0['dealdate']]
df.at[20793, 'testTend']=-df.at[20793, 'testTend']
df.at[20792, 'testTend']=-df.at[20792, 'testTend']
df.at[20791, 'testTend']=-df.at[20791, 'testTend']
df.at[20790, 'testTend']=-df.at[20790, 'testTend']
df.at[20789, 'testTend']=-df.at[20789, 'testTend']
df.at[20788, 'testTend']=-df.at[20788, 'testTend']
calculatePL()

In [0]:

for index, row in df.iterrows():
    if index>=20789 and index<=20793:
        df.at[index, 'testTend']=1#-df.at[index, 'testTend']
    '''
    df.at[20792, 'testTend']=-df.at[20792, 'testTend']
    df.at[20791, 'testTend']=-df.at[20791, 'testTend']
    df.at[20790, 'testTend']=-df.at[20790, 'testTend']
    df.at[20789, 'testTend']=-df.at[20789, 'testTend']
    df.at[20788, 'testTend']=-df.at[20788, 'testTend']
    '''
calculatePL()
df[df['index']<=20793].tail()

In [0]:
dftmp0=df[df['inventoryPLmin']==-84].head(1)
print(dftmp0)
df.drop(["open","high","low","volume",'dealdateI','dealtimeI','testTend','baseprice'], axis=1).tail()

In [0]:

print(datetime.datetime.now(tz))  
dfNext=df.drop(["tendMajor","tendMinor","baseprice","inventoryPL","inventoryPLmin","settlePL","settlePLmin","settlePLsum"], axis=1)

for index, row in dfNext.iterrows():
    #dfNext.at[index, 'lastclose']=df2[ df2["dealdate"]==row["dealdate"] ]["lastclose"]
    df3=df2[ df2['dealdate']==row['dealdate'] ] #first dealtime when low hit lowest
    dfNext.at[index,'lastclose']=df3['lastclose'].iloc[0]
print(dfNext.head())
print(dfNext.tail())
print(datetime.datetime.now(tz))  

接下來應該要用到的欄位：

1. dealdate
2. dealtime
3. lastclose (前一交易日日盤)
4. X preclose (夜盤收盤, 目前有困難)
5. open
6. high
7. low
8. close
9. volume
10. rsi ? if any



In [0]:

print(datetime.datetime.now(tz))  
dfNext['date'] = pd.to_datetime(dfNext['dealdatetime'])
dfNext['close'] = pd.to_numeric(dfNext['close'], downcast='float')
dfNext.dtypes


In [0]:
!pip install stockstats
import stockstats
from stockstats import StockDataFrame as Sdf

stock = Sdf.retype(dfNext)

print(datetime.datetime.now(tz))  

In [0]:

dfNext['macd']=stock['macd']
dfNext['rsi_6']=stock['rsi_6']
dfNext['rsi_12']=stock['rsi_12']
dfNext['rsi_14']=stock['rsi_14']
dfNext['kdjk']=stock['kdjk']
dfNext['kdjd']=stock['kdjd']
dfNext['kdjj']=stock['kdjj']
dfNext['boll']=stock['boll']
dfNext['boll_ub']=stock['boll_ub']
dfNext['boll_lb']=stock['boll_lb']
dfNext['wr_10']=stock['wr_10']
dfNext['wr_6']=stock['wr_6']
dfNext['dma']=stock['dma']
dfNext['pdi']=stock['pdi']
dfNext['mdi']=stock['mdi']
dfNext['dx']=stock['dx']
dfNext['adx']=stock['adx']
dfNext['adxr']=stock['adxr']
dfNext['vr']=stock['vr']
dfNext['vr_6_sma']=stock['vr_6_sma']

#dfNext['close_-1_d']=dfNext['lastclose']
#dfNext["rsi_6"]=stock["rsi_6"]
#dfNext=dfNext.drop(["close_12_ema","close_26_ema","close_-1_s"], axis=1)
#dfNext=dfNext.drop(["close_-1_d"], axis=1)
print(datetime.datetime.now(tz))  
dfNext.to_csv('dfNext.csv', mode='a', header=True, index=False)
df2.to_csv('df2.csv', mode='a', header=True, index=False)
print(dfNext.dtypes)
dfNext.head(5)

# 可以從這裡開始重複

In [0]:
import pandas as pd
from collections import deque
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint
import time
from sklearn import preprocessing
from datetime import datetime
import datetime
import io
import sys
#sys.exit()
print(datetime.datetime.now(tz))

#PERCENT=0.2 # normaly 0.067 for a whole year
SEQ_LEN = 20  # how long of a preceeding sequence to collect for RNN

In [0]:
#may continue from here without rerun

#這邊搞定nan與infinite
dfNext=pd.read_csv('dfNext.csv')
df2=pd.read_csv('df2.csv')

dfNext=dfNext.replace([np.inf, -np.inf], np.nan) # replace all inf (infinite) with nan
dfNext['close_-1_s'].fillna(dfNext['close'], inplace=True)
dfNext['close_-1_d'].fillna(0, inplace=True)
dfNext['rs_6'].fillna(0, inplace=True)
dfNext['rsi_6'].fillna(100, inplace=True)
dfNext['rs_12'].fillna(0, inplace=True)
dfNext['rsi_12'].fillna(100, inplace=True)
dfNext['rs_14'].fillna(0, inplace=True)
dfNext['rsi_14'].fillna(100, inplace=True)
dfNext['close_20_mstd'].fillna(0, inplace=True)
dfNext['boll_ub'].fillna(dfNext['close'], inplace=True)
dfNext['boll_lb'].fillna(dfNext['close'], inplace=True)
dfNext['high_delta'].fillna(0, inplace=True)
dfNext['um'].fillna(0, inplace=True)
dfNext['low_delta'].fillna(0, inplace=True)
dfNext['dm'].fillna(0, inplace=True)
dfNext['tr'].fillna(0, inplace=True)
dfNext['atr_14'].fillna(0, inplace=True)
dfNext['pdi_14'].fillna(0, inplace=True)
dfNext['pdi'].fillna(0, inplace=True)
dfNext['mdi_14'].fillna(0, inplace=True)
dfNext['mdi'].fillna(0, inplace=True)
dfNext['dx_14'].fillna(100, inplace=True)
dfNext['dx'].fillna(100, inplace=True)
dfNext['dx_6_ema'].fillna(100, inplace=True)
dfNext['adx'].fillna(100, inplace=True)
dfNext['adx_6_ema'].fillna(100, inplace=True)
dfNext['adxr'].fillna(100, inplace=True)
dfNext['change'].fillna(0, inplace=True)
dfNext['vr'].fillna(100, inplace=True)
dfNext['vr_6_sma'].fillna(100, inplace=True)
#dfNext.loc[:, dfNext.isna().any()]
print(dfNext.columns[dfNext.isna().any()].tolist())
#print(dfNext[dfNext.isnull().any(axis=1)])

print(dfNext.dtypes)
dfNext.head()

In [0]:
#dfNext['dealdate']=dfNext['dealdate'].astype('|S')
#dfNext['dealtime']=pd.to_string(dfNext['dealtime'])
#dfNext['dealdatetime']= pd.to_datetime(dfNext['dealdatetime'])

#dfNext['dealdate']=dfNext['dealdate'].replace("-","")
#dfNext['dealtime']=dfNext['dealtime'].replace(":","")

#dfNext['dealdate'] = pd.to_numeric(dfNext['dealdate'], downcast='float')
#dfNext['dealtime'] = pd.to_numeric(dfNext['dealtime'], downcast='float')

#dfNext['dealdatetimeL']=dfNext['dealdatetime'].strftime("%Y%m%d%H%M%S")
#dfNext[['dealdate','dealtime','dealdatetime','dealdatetimeL']].head()
print(dfNext[['dealdate','dealtime','dealdatetime']].dtypes)
dfNext[['dealdate','dealtime','dealdatetime']].head()

好，接下來就從這裡用dfNext來進行LSTM的分析


0.   刪除
1. 分兩段df, training 與 verify, verify 取10 or 15天
2.   寫preprocess
  a. 把所有欄位值調整到0 to 1
  b. 利用deque將資料調為一段一段, 一段20根k棒
  c. 每20K棒產生一個趨勢，然後產生一堆所有如此的K棒
  d. randomize
  e. return ndarray 與 已知正確趨勢
3. 產生model
4. 最佳化
5. 比對verification set data
  



In [0]:
# training data and verification data separated

validation_dealdate=df2.tail(10)['dealdate'].min()

validation_dfNext = dfNext[(dfNext.dealdate >= validation_dealdate)]
dfNext = dfNext[(dfNext.dealdate < validation_dealdate)]

#validation_dfNext

# why testtend contains nan??!!
print(dfNext.columns[dfNext.isna().any()].tolist())
print(validation_dfNext.columns[validation_dfNext.isna().any()].tolist())
#dfNext.dtypes
#validation_dfNext[['dealdatetime','testtend']]
#print(validation_dfNext['testtend'].values.min())
#print(validation_dfNext['testtend'].values.max())

In [0]:

#dfNext
def preprocess_df(df): # df: input dataframe
    
    df["predictedTrend"]=0  # don't need this anymore.
    df=df.drop(['dealdate','dealtime','dealdatetime'], axis=1)
    for col in df.columns:  # go through all of the columns
        if col != "predictedTrend":  # normalize all ... except for the target itself!
            df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
            #df.dropna(inplace=True)  # remove the nas created by pct_change
            df=df.replace([np.inf, -np.inf], np.nan) # replace all inf (infinite) with nan
            df.fillna(0, inplace=True)
            
            #print(df.columns[df.isna().any()].tolist())
            #print(col)
            
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.
        if col == "dealtimei":
            print(df[col].values)
            print(df[col].shape)
    #print(df.columns[df.isna().any()].tolist())
    print(df.head())
    print(df.shape)
    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in
    l=0
    for i in df.values:  # iterate over the values, each i become a sub array to be appended, this is to convert from dataframe to ndarray
        # i is df row, without index (time here)
        prev_days.append([n for n in i[:-1]])  # store all but the target
        '''
        if l<5:
            print([np.array(prev_days), i[-1]])
        '''
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!
        l=l+1
    print(prev_days) #very painful, deque cannot print anything but all
    #print(len(sequential_data))
    random.shuffle(sequential_data)  # another shuffle, so the model doesn't get confused with all 1 class then the other.
    
    X = []
    y = []
    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)
    return np.array(X), y  # return X and y...and make X a numpy array!
validation_x, validation_y = preprocess_df(validation_dfNext)
print(len(validation_x[0]))
print(validation_x[0][0])
print(validation_y[0])

In [0]:
#df2